In [3]:
#Importing Packages

# Importing Pandas for data manipulation and analysis
import pandas as pd

# Importing NumPy for numerical operations
import numpy as np

# Importing Matplotlib for data visualization
import matplotlib.pyplot as plt

# Importing Seaborn for advanced data visualization
import seaborn as sns

# Importing IPython.display for displaying rich media in Jupyter Notebooks
from IPython.display import display, Image


<h3 style=\"color:green;\">DATA LOADING</h3>
    
    Firstly, import the necessary packages. When the required packages are imported, for loading a CSV file, this Project will utilise Pandas to load the data.

In [4]:
# Load the CSV file
train_df = pd.read_csv('Data/processed/train.csv')
test_df = pd.read_csv('Data/processed/test.csv')

In [5]:
train_df.head()

,headlines,description,content,url,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,https://indianexpress.com/article/business/ban...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,https://indianexpress.com/article/business/com...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,https://indianexpress.com/article/business/mar...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,https://indianexpress.com/article/business/eco...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,https://indianexpress.com/article/business/eco...,business


<h3 style=\"color:green;\">PRE-PROCESSING</h3>
    
- Remove unnecessary columns (url) since it may not contribute to categorization.
- *Text Cleaning*: Convert text to lowercase, remove punctuation, stopwords, and special characters.
- *Tokenization*: Split text into individual words or phrases.
- *Vectorization*: Convert text into numerical form using TF-IDF or CountVectorizer.

In [13]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

#Remove unnecessary columns
#train_df = train_df.drop('url', axis=1)
#test_df = test_df.drop('url', axis=1)

#Text Cleaning
#Convert text to lowercase
train_df = train_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

display(train_df)


ModuleNotFoundError: No module named 'nltk'

In [8]:
train_df.head()
test_df.head()

,headlines,description,content,category
0,NLC India wins contract for power supply to Ra...,State-owned firm NLC India Ltd (NLCIL) on Mond...,State-owned firm NLC India Ltd (NLCIL) on Mond...,business
1,SBI Clerk prelims exams dates announced; admit...,SBI Clerk Prelims Exam: The SBI Clerk prelims ...,SBI Clerk Prelims Exam: The State Bank of Indi...,education
2,"Golden Globes: Michelle Yeoh, Will Ferrell, An...","Barbie is the top nominee this year, followed ...","Michelle Yeoh, Will Ferrell, Angela Bassett an...",entertainment
3,"OnePlus Nord 3 at Rs 27,999 as part of new pri...",New deal makes the OnePlus Nord 3 an easy purc...,"In our review of the OnePlus Nord 3 5G, we pra...",technology
4,Adani family’s partners used ‘opaque’ funds to...,Citing review of files from multiple tax haven...,Millions of dollars were invested in some publ...,business
